In [45]:
import pandas as pd
import numpy as np

import re

# visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': small,
          'figure.figsize': (14, 9),
          'axes.labelsize': small,
          'axes.titlesize': small,
          'xtick.labelsize': small,
          'ytick.labelsize': small,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

%matplotlib inline

In [141]:
# Загружаем БД:
df = pd.read_excel('data/data.xlsx')

In [142]:
# Меняем названия на более приемлемые:

target_shops = [
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"', 'ООО "АГРОТОРГ"', 'ООО "ЛЕНТА"',
   'АО "ДИКСИ ЮГ"', 'ООО "АШАН"', 'АО "ТАНДЕР"', 'ООО "БИЛЛА"', 'ООО "О`КЕЙ"',
    'ООО "АГРОАСПЕКТ"', 'ООО "АТАК"'
]

shops_rename = {
    'АО "ТОРГОВЫЙ ДОМ "ПЕРЕКРЕСТОК"': 'ПЕРЕКРЕСТОК',
    'ООО "АГРОТОРГ"': 'ПЯТЕРОЧКА',
    'ООО "ЛЕНТА"': 'ЛЕНТА',
    'АО "ДИКСИ ЮГ"': 'ДИКСИ',
    'ООО "АШАН"': 'АШАН',
    'АО "ТАНДЕР"': 'МАГНИТ',
    'ООО "БИЛЛА"': 'БИЛЛА',
    'ООО "О`КЕЙ"': 'О`КЕЙ',
    'ООО "АГРОАСПЕКТ"': 'ПЯТЕРОЧКА',
    'ООО "АТАК"': 'АТАК'
}

df = df[df['shop_name'].isin(target_shops)].copy()
df['shop_name'] = df['shop_name'].apply(lambda x: shops_rename[x])
df.sample()

,shop_name,name,quantity,price,sum
32810,ПЯТЕРОЧКА,809 Бананы 1кг,0.435,65.99,28.71
27233,ПЯТЕРОЧКА,3618224 Томаты КОКТЕЙЛЬНЫЕ 1кг,0.615,218.49,134.37
27764,ПЯТЕРОЧКА,809 Бананы 1кг,1.115,59.99,66.89
39703,ПЯТЕРОЧКА,*45625 LIBRES.Прок.ULTRA НОРМАЛ 20шт,1.000,129.99,129.99
34442,ПЯТЕРОЧКА,3497593 ЩЕДР.Хлебцы тонкие ржаные 170г,1.000,71.99,71.99


In [145]:
# Выбираем только магазин ПЯТЕРОЧКА:
df = df[df['shop_name'] == 'ПЯТЕРОЧКА'].drop_duplicates('name').reset_index(drop=True)
df.head()

,shop_name,name,quantity,price,sum
0,ПЯТЕРОЧКА,*3074007 Морковь мытая 1кг,0.71,55.99,39.75
1,ПЯТЕРОЧКА,3248992 КРАСНАЯ ЦЕНА Огурцы марин.720мл,1.00,63.59,63.59
2,ПЯТЕРОЧКА,*19134 GR.R.Горошек зеленый ж/б 425мл,1.00,54.99,54.99
3,ПЯТЕРОЧКА,3334702 Ч.ЛИН.Морож.плом.ван.12% 450г,1.00,339.99,339.99
4,ПЯТЕРОЧКА,*3696849 ПР.НАСТР.Прян.с шок/ап/лим.230г,1.00,59.99,59.99


In [146]:
# Смотрим, в каком виде существуют данные:
print('\n'.join(df['name'].head(20).values))

*3074007 Морковь мытая 1кг
3248992 КРАСНАЯ ЦЕНА Огурцы марин.720мл
*19134 GR.R.Горошек зеленый ж/б  425мл
3334702 Ч.ЛИН.Морож.плом.ван.12% 450г
*3696849 ПР.НАСТР.Прян.с шок/ап/лим.230г
3413519 БР-ЛИТ.Масло сл/сл.в/с82,5%180г
*3463717 HAAS Ароматиз.ВАНИЛИН пищ.1,5г
*4038313 DR.TR.Игр-карт.Б.ГАСТ.д/н.игры
*3371481 Нап.пив.KRONENB.1664БЛ.0.46л
*3141246 Напиток Черноголовка Байкал 2л
3437426 Грунт универсальный 5л
3300573 Пакет ПЯТЕРОЧКА 65х40см
*2157107 COLG.З/п МАКС БЛЕСК 100мл
3380551 IN.Сол.МОРС.пищ.м/м мел/п 0,5кг
*88780 KOTANYI Перец черный мол,пак20г
2250692 Дск Огурцы короткоплодные 450г
99593 ТРИ КОРОЧ.Сух.рж.холод/хрен 40г
*3385945 KOTANYIПриправаИТАЛ.ТРАВЫпак14г
*88781 KOTANYI Чеснок изм.пак 28г
*3409453 СИГАРЕТЫ BOND ST.COMP.BLUE


In [147]:
# Избавляемся от цифр в начале:
df['name'] = df['name'].apply(lambda x: ' '.join(x.split()[1:]))
df.head()

,shop_name,name,quantity,price,sum
0,ПЯТЕРОЧКА,Морковь мытая 1кг,0.71,55.99,39.75
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин.720мл,1.00,63.59,63.59
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б 425мл,1.00,54.99,54.99
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12% 450г,1.00,339.99,339.99
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим.230г,1.00,59.99,59.99


#### Работаем с весом, объёмом товара или количеством:

In [161]:
# Находим вес или объём товара:

def find_weight_or_volume(name: str)-> str:
    """Находит вес или объём товара."""

    pattern1 = r'\d+((\.|\,|x|X|х|Х)?)\d*([а-я]|[А-Я])*$' # ищет вес в конце
    pattern2 = r'\d+\w+' #  ищет вес в середине
    pattern3 = r'(\d+ кг)|(\d+ г)'  # находит `1 кг` или `1 г`
    result = re.search(pattern1, name)
    if result:
        return result.group(0)
    result = re.search(pattern2, name)
    if result:
        return result.group(0)
    result = re.search(pattern3, name)
    return result.group(0) if result else ''
    

df['size'] = df['name'].apply(find_weight_or_volume)
df.head()

,shop_name,name,quantity,price,sum,size
0,ПЯТЕРОЧКА,Морковь мытая 1кг,0.71,55.99,39.75,1кг
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин.720мл,1.00,63.59,63.59,720мл
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б 425мл,1.00,54.99,54.99,425мл
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12% 450г,1.00,339.99,339.99,450г
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим.230г,1.00,59.99,59.99,230г


In [162]:
# Смотрим, какие записи не получилось распознать:
print('\n'.join(df[df['size'] == '']['name'].values))

DR.TR.Игр-карт.Б.ГАСТ.д/н.игры
СИГАРЕТЫ BOND ST.COMP.BLUE
САДОК Сем.Пет.Хор.нас.F1 см.окр
СИГАРЕТЫ LD AUT.IMP.COMP.
Зелень свежая в ассортименте
СИГАРЕТЫ WINSTON BLUE
РОСК.РОС.Наб.пасх.ЗАБАВ.ЖИВОТН.
СИГАРЕТЫ SOBRANIE СИНИЕ
СИГАРЕТЫ WINSTON WHITE
АРТ И ДИЗАЙ.Сумка EXLARGE плас.
РОСК.РОС.Наб.пасх.ВЕСЕЛ.ЖИТЕЛИ
СИГАРЕТЫ PHILIP MOR.COMP.BL.
Напиток миндальный обогащ кальц
ECON.Рожок д/обуви
СИГАРЕТЫ MARLBORO COMPACT
Напиток кокосовый с рисом обог
САДОК Семена Шпинат Матадор
САДОК Сем.Горох ов.Детский сах.
САДОК Сем.Горох овощ.Сах.принц
СИГАРЕТЫ ЯВА ЗОЛОТАЯ ОРИГИН.
СИГАРЕТЫ LD AUT.CLUB PLAT.
РОСК.РОС.Наб.пасх.ПУШИСТИКИ
РОСК.РОС.Наб.пасх.ПУШИСТ.УТЯТА
СИГАРЕТЫ PARLIAMENT SILVER BLUE
Пельмени Останкинские Традицион
СИГАРЕТЫ ДОНСКОЙ ТАБАК СВ.
ВОР.Сух.ГРИБЫ СО СМЕТАНЕ рж-пш
ECONTA Фольга алюминиевая бытов
Газета КОМСОМОЛЬСК.ПРАВДА ежен
DIROL Жев.рез.WHITE Клуб.поляна
СИГАРЕТЫ LD AUTOGRAPH RED
COLG.З/щ МАССАЖЕР ср/ж
Кн.Футбол с наклейками
СИГАРЕТЫ KENT БЛЮ
РОСК.РОС.Наб.пасх.ВЕСЕН.БАБОЧКИ
СИГАРЕТЫ 

##### Вроде бы получилось распознать почти всё

In [170]:
def remove_size_from_name(name: str, size: str):
    '''Удаляет из названия товара цену.'''

    return name.replace(size, '').strip().strip('.')

df[['name']] = df[['name', 'size']].apply(lambda x: remove_size_from_name(**x), axis=1)
df.head()

,shop_name,name,quantity,price,sum,size
0,ПЯТЕРОЧКА,Морковь мытая,0.71,55.99,39.75,1кг
1,ПЯТЕРОЧКА,КРАСНАЯ ЦЕНА Огурцы марин,1.00,63.59,63.59,720мл
2,ПЯТЕРОЧКА,GR.R.Горошек зеленый ж/б,1.00,54.99,54.99,425мл
3,ПЯТЕРОЧКА,Ч.ЛИН.Морож.плом.ван.12%,1.00,339.99,339.99,450г
4,ПЯТЕРОЧКА,ПР.НАСТР.Прян.с шок/ап/лим,1.00,59.99,59.99,230г
